# Descriptor

### 학습 내용
- descriptor가 무엇인지 이해하고 어떻게 효율적으로 동작하는지 이해한다.
- data descriptor vs non-data descriptor 의 개념적 차이와 세부구현의 차이를 분석
- descriptor를 활용한 코드 재사용 방법
- 좋은 활용 예

학습에 앞서 먼저 descriptor의 syntax를 알아보고자 한다.
https://docs.python.org/3/howto/descriptor.html<br>
위 사이트에서 공부하였음

가장 쉬운 예제는 아래와 같다<br>
A descripotr that returns a constant

In [36]:
class Ten:
    def __get__(self, obj, objtype=None):
        return 10

To use the desciptor, it must be stored as a class Variable in another class<br>

In [37]:
class A:
    x = 5
    y = Ten()

In [38]:
a = A()
print(a.x)
print(a.y)

5
10


In the a.x attribute lookup, the dot operator finds the key x<br>
and the value 5 in the class dictionary.<br>

In the a.y lookup, the dot operator finds a descripotr instance, <br>
recognized by its \_\_get__ method, and calls that method which returns 10<br>

Note that the value 10 is not stored in either the class or the instance dic<br>
Instead, the value 10 in computed on demand

and we're gonna see dynamic lookup

Interesting descriptor typically run computation instead of returning constant<br>

In [39]:
import os

class DirectorySize:
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

class Directory:
    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname

In [40]:
s = Directory('../')
j = Directory('../joono')

print(s.size)
print(j.size)

9
8


위와 같은 예제를 살펴보면 \_\_get__ method가 어떻게 동작하는지, 그 목적이 무엇인지 알 수 있다.<br>

descriptor가 가장 자주 사용되는 부분은 instance의 data에 접근을 제어하는 일이다.<br>
Descriptor는 public attribute에 저장되고 실제 data는 다른 instance dict에 저장된다.<br>
그리고 descriptor의 \_\_get__, \_\_set__ methods는 public attribute에 접근할 때 호출된다.<br>

아래의 예제에서 _age_는 public attribute,<br>
_\_age_는 private attribute이다.<br>
그리고 age에 access할 때 마다 descriptor가 lookup, update를 log한다.

In [41]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAgeAccess:

    def __get__(self, obj, objtype=None):
        value = obj._age
        logging.info('Accessing %r giving %r', 'age', value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', 'age', value)
        obj._age = value

class Person:

    age = LoggedAgeAccess()             # Descriptor instance

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1

아래 결과를 보면 regular attribute인 name에 access할 때는 아무런 log도 남지 않지만<br>
Descriptor로 정의된 age에 접근할 때면 desciptor가 log를 남긴다.

_obj_ parameter는 g나 s같은 instance of Dictionary이고 이는 target directory가 무엇인지<br>
\_\_get__ method 에게 알려주는 역할을 한다.<br>
_objtype_ parameter는 class Dictionary이다.<br>

In [42]:
mary = Person('Mary M', 30)
david = Person('David D', 40)

INFO:root:Updating 'age' to 30
INFO:root:Updating 'age' to 40


In [43]:
print(vars(mary))
print(vars(david))

{'name': 'Mary M', '_age': 30}
{'name': 'David D', '_age': 40}


In [44]:
mary.age

INFO:root:Accessing 'age' giving 30


30

In [45]:
david.age

INFO:root:Accessing 'age' giving 40


40

그러나 위와 같은 방법의 문제점은 딱 age 하나 밖에 descriptor를 설정하지 못한다는 단점이 있다.
아래에서 위와 같은 문제를 해결해보자

descriptor를 사용할 때 각 descriptor에게 어떤 변수명이 사용됐는지 알려줄 수 있다.<br>
_Person_ class는 2개의 name, age라는 descriptor instance를 가지고 있다.<br>

_Person_ class가 정의될 때, \_\_set_name__이라는 함수를 callback한다.<br>
name, age에게 이들의 public, private 변수명을 알려주기 위해서이다.

In [46]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name

    def __get__(self, instance, owner):
        value = getattr(instance, self.private_name)
        logging.info(f'Accessing {self.public_name} to {value}')
        return value

    def __set__(self, instance, value):
        logging.info(f'Updating {self.public_name} to {value}')
        setattr(instance, self.private_name, value)

class Person:
    name = LoggedAccess()
    age = LoggedAccess()

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def birthday(self):
        self.age += 1

Person classr가 정의될 때, 각 public, private name들을 저장하기 위해 \_\_set_name__이 callback 되었다.<br>
vars 함수를 통해 정말 그렇게 되었는지 확인해보자.<br>

descriptor는 'dot operator'에 의해서만 invoke 된다.<br>
따라서 아래의 두 코드에서는 log가 발생되지 않는다.

In [47]:
vars(vars(Person)['name'])

{'public_name': 'name', 'private_name': '_name'}

In [48]:
vars(vars(Person)['age'])

{'public_name': 'age', 'private_name': '_age'}

그러나 아래의 코드에서는 모두 dot operator가 사용되므로 log가 발생했다.

In [49]:
pete = Person('Pete P', 10)
kate = Person('Kate K', 20)

INFO:root:Updating name to Pete P
INFO:root:Updating age to 10
INFO:root:Updating name to Kate K
INFO:root:Updating age to 20


descriptor instance에는 private name만 contain하게 된다.

In [50]:
print(vars(pete))
print(vars(kate))

{'_name': 'Pete P', '_age': 10}
{'_name': 'Kate K', '_age': 20}


## Closing thought
- descriptor는 \_\_get__, \_\_set__, \_\_delete__ method를 정의하는 class를 지칭하며,<br>
optional 하게 \_\_set_name__ 도 포함할 수 있다.
- descriptor는 dot operator에 의해서만 invoke 된다.
- 오직 class variable로 선언될 때만 사용 가능하다. instance로 선언되면 아무런 효과도 없다.
- descriptor의 motivation은 attrubute를 lookup할 때 발생하는 상황에 대하여 통제할 수 있도록<br>
 class variable에 objects를 저장할 수 있는 hook을 제공하는 것이다.<br>
- classmethod, staticmethod, property등등이 descriptor로서 사용된다.

- 아래는 본문에서 제시한 유효성을 검사하는 객체를 어떻게 만드는지 보여주는 예이다.
- 유효성 검사 함수는 자유롭게 생성할 수 있으며, 객체에 값을 할당하기 전에 실행된다.

In [51]:
class Validation:
    def __init__(self, validation_func, error_msg:str):
        self.validation_func = validation_func
        self.error_msg = error_msg

    def __call__(self, value):
        if not self.validation_func(value):
            raise ValueError(f'{value!r} {self.error_msg}')

class Field:    # descriptor
    def __init__(self, *validations):
        self._name = None
        self.validations = validations

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        if instance is None:    # classㄴ에서 호출
            return self
        return instance.__dict__[self._name]

    def validate(self, value):
        for validation in self.validations:
            validation(value)

    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), '는 숫자가 아님'),
        Validation(lambda x: x >= 0, "는 0보다 작음")
    )

In [52]:
client = ClientClass()
client.descriptor = 42
client.descriptor

42

In [53]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [54]:
client.descriptor = 'invalid value'

ValueError: 'invalid value' 는 숫자가 아님

여기서 기억할 점은 property 자리에 놓일 수 있는 것은 descriptor로 추상화할 수 있으며<br>
여러번 재사용이 가능하다는 것이다.

다음 예제는 이 메서드를 사용하여 관리자 권한이 없는 객체에서 속성을 제거하지 못하도록 하는 디스크립터를 만든다.<br>

In [56]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, user, value):
        if value is None:
            raise ValueError(f'{self._name}를 None으로 설정할 수 없음')
        user.__dict__[self._name] = value

    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(f'{user!r} 사용자는 {self.permission_required} 권한이 없음')

class User:
    """admin 권한을 가진 사용자만 이메일을 주소를 삭제할 수 있음."""
    email = ProtectedAttribute(requires_role="admin")

    def __init__(self, user_name: str, email: str, permissions: list=None) -> None:
        self.username = user_name
        self.email = email
        self.permissions = permissions or []

    def __str__(self):
        return self.username

In [61]:
admin = User('root', 'root@d.com', ['admin'])
user = User('user', 'user1@d.com', ['email', 'helpdesk'])
admin.email

'root@d.com'

In [63]:
del admin.email
admin.email is None

True

In [64]:
user.email

'user1@d.com'

In [65]:
del user.email

ValueError: <__main__.User object at 0x7ffa60529d00> 사용자는 admin 권한이 없음

### \_\_set_name__
- 일반적으로 하위 호환을 위해 \_\_init__에 기본 값을 지정하고 \_\_set_name__ 를 함께 사용하는 것이 좋다

# descriptor의 유형

- data descrptor : \_\_set__이나 \_\_delete__ method를 구현함
- non-data descriptor : \_\_get__만 구현함
- \_\_set_name__ 은 위 분류에 영향을 미치지 않는다.

### non-data descriptor

In [67]:
class NonDataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42

class ClientClass:
    descriptor = NonDataDescriptor()

In [68]:
client = ClientClass()
client.descriptor

42

In [69]:
client.descriptor = 43
client.descriptor

43

여기서 descriptor를 지우고 다시 lookup하게 되면 어떤 결과를 얻게 될까?

In [70]:
del client.descriptor
client.descriptor

42

In [71]:
vars(client)

{}

.descriptor attribute을 lookup하면 client.\_\_dict__에서 'descriptor'를 찾지 못하고<br>
마침내 클래스에서 디스크립터를 찾게 된다. 이것이 \_\_get__ method가 호출되는 이유이다.

그러나 \_\_set__ method가 정의되지 않으면 인스턴스의 사전이 변경되므로 client.\_\_dict__ 가 변경된다.<br>

In [72]:
client.descriptor = 99
vars(client)

{'descriptor': 99}

### data descriptor
data descriptor와의 차이를 살펴보기 위해 새로운 클래스를 정의하자